<a href="https://colab.research.google.com/github/cesarcamp/NLP-with-Disaster-Tweets/blob/master/NLP_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test (2).csv
Saving train.csv to train (2).csv


In [3]:
import pandas as pd
import regex as re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [4]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [11]:
english_vocab = set(w.lower() for w in nltk.corpus.words.words())

##DATA CLEANING

In [19]:
def clean_up(s):
    s=re.sub(r'http\S+',' ',s)
    s=re.sub('\W', ' ',s)
    s=re.sub('\d', ' ',s)
    s=re.sub("\s+"," ",s)
    s=s.lower().strip()
    s=" ".join([i for i in s.split() if i in english_vocab])
    return s

In [20]:
def tokenize_lemmatize(s):
    s=clean_up(s)
    s=word_tokenize(s)
    lst = []
    for token in s:
        lst.append(WordNetLemmatizer().lemmatize(token))
    return " ".join(lst)

In [28]:
train.text = train.text.apply(lambda x: tokenize_lemmatize(x))
test.text = test.text.apply(lambda x: tokenize_lemmatize(x))

##VECTORIZING

In [29]:
vectorizertfidf = TfidfVectorizer(stop_words='english')

In [30]:
X=train["text"]
y=train["target"]

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.01, random_state = 42)

In [32]:
X_train_bow = vectorizertfidf.fit_transform(X_train)
X_test_bow = vectorizertfidf.transform(X_test)

# ML MODELS


In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB as GNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier as XGBC
from sklearn.ensemble import GradientBoostingClassifier as GBC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [40]:
gbc = GBC()
gbc.fit(X_train_bow, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [81]:
gbc_accuracy_test = gbc.score(X_test_bow, y_test)
gbc_accuracy_train = gbc.score(X_train_bow, y_train)
print("gbc_accuracy_test : {}".format(gbc_accuracy_test)) 
print("gbc_accuracy_train : {}".format(gbc_accuracy_train))

gbc_accuracy_test : 0.7662337662337663
gbc_accuracy_train : 0.7562367303609342


In [43]:
mnb=MultinomialNB()
mnb.fit(X_train_bow, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [82]:
mnb_accuracy_test = mnb.score(X_test_bow, y_test)
mnb_accuracy_train = mnb.score(X_train_bow, y_train)
print("mnb_accuracy_test : {}".format(mnb_accuracy_test)) 
print("mnb_accuracy_train : {}".format(mnb_accuracy_train))

mnb_accuracy_test : 0.8051948051948052
mnb_accuracy_train : 0.8631900212314225


In [45]:
xgbc=XGBC()
xgbc.fit(X_train_bow, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [83]:
xgbc_accuracy_test = xgbc.score(X_test_bow, y_test)
xgbc_accuracy_train = xgbc.score(X_train_bow, y_train)
print("xgbc_accuracy_test : {}".format(xgbc_accuracy_test)) 
print("xgbc_accuracy_train : {}".format(xgbc_accuracy_train))

xgbc_accuracy_test : 0.7662337662337663
xgbc_accuracy_train : 0.7384554140127388


In [47]:
rfc = RandomForestClassifier()

In [48]:
rfc.fit(X_train_bow, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [84]:
rfc_accuracy_test = rfc.score(X_test_bow, y_test)
rfc_accuracy_train = rfc.score(X_train_bow, y_train)
print("rfc_accuracy_test : {}".format(rfc_accuracy_test)) 
print("rfc_accuracy_train : {}".format(rfc_accuracy_train))

rfc_accuracy_test : 0.8051948051948052
rfc_accuracy_train : 0.9785031847133758


In [51]:
svc=SVC()
svc.fit(X_train_bow, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [85]:
svc_accuracy_test = svc.score(X_test_bow, y_test)
svc_accuracy_train = svc.score(X_train_bow, y_train)
print("svc_accuracy_test : {}".format(svc_accuracy_test)) 
print("svc_accuracy_train : {}".format(svc_accuracy_train))

svc_accuracy_test : 0.8051948051948052
svc_accuracy_train : 0.9506369426751592


In [53]:
lgc = LogisticRegression()

In [54]:
lgc.fit(X_train_bow, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [86]:
lgc_accuracy_test = lgc.score(X_test_bow, y_test)
lgc_accuracy_train = lgc.score(X_train_bow, y_train)
print("gbc_accuracy_test : {}".format(lgc_accuracy_test)) 
print("gbc_accuracy_train : {}".format(lgc_accuracy_train))

gbc_accuracy_test : 0.7662337662337663
gbc_accuracy_train : 0.8577494692144374


##GRID SEARCHING

In [77]:
# defining parameter range 
param_grid = {'C': [0.1, 1, 10, 100, 1000],  
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
              'kernel': ['rbf']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3) 
grid.fit(X_train_bow, y_train) 

Fitting 5 folds for each of 25 candidates, totalling 125 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.611, total=   3.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.8s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.613, total=   3.7s
[CV] C=0.1, gamma=1, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    7.5s remaining:    0.0s


[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.621, total=   3.7s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.612, total=   3.8s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] .......... C=0.1, gamma=1, kernel=rbf, score=0.622, total=   3.7s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.570, total=   3.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.569, total=   3.6s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.570, total=   3.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] ........ C=0.1, gamma=0.1, kernel=rbf, score=0.570, total=   3.5s
[CV] C=0.1, gamma=0.1, kernel=rbf ....................................
[CV] .

[Parallel(n_jobs=1)]: Done 125 out of 125 | elapsed:  7.5min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['rbf']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [87]:
grid_predictions = grid.predict(X_test_bow)

In [90]:
grid_accuracy_test = grid.score(X_test_bow, y_test)
grid_accuracy_train = grid.score(X_train_bow, y_train)
print("grid_accuracy_test : {}".format(grid_accuracy_test)) 
print("grid_accuracy_train : {}".format(grid_accuracy_train))

grid_accuracy_test : 0.8051948051948052
grid_accuracy_train : 0.9497080679405521


##MODEL SUMMARY

In [95]:
models = {"Model": {0:"Logistic Regression", 1:"SVC", 2:"RandomForestClassifier", 3:"XGBClassifier", 
                    4:"GradientBoostingClassifier", 5:"MultinomialNB", 6:"Grid Searching"}, 
          "Score": {0:lgc_accuracy, 1:svc_accuracy_test, 2:rfc_accuracy_test, 3:xgbc_accuracy_test, 
                    4:gbc_accuracy_test, 5:mnb_accuracy_test, 6:grid_accuracy_test}}

In [98]:
models = pd.DataFrame(models)

In [101]:
models = models.sort_values(by=["Score"], ascending=False)

In [102]:
models

,Model,Score
1,SVC,0.805195
2,RandomForestClassifier,0.805195
5,MultinomialNB,0.805195
6,Grid Searching,0.805195
0,Logistic Regression,0.766234
3,XGBClassifier,0.766234
4,GradientBoostingClassifier,0.766234


##BEST MODEL SUBMISSION

In [103]:
test["sentiment"] = test["text"].apply(lambda x : svc.predict(vectorizertfidf.transform([x])))

In [104]:
def number(x):
    a=0
    for i in x:
        a+=i
    return a 

In [105]:
test["target"] = test["sentiment"].apply(lambda x : number(x))

In [106]:
submission = test[["id","target"]]

In [107]:
submission.to_csv("svc_8051.csv", index= False)